# Курс "Программирование на языке Python. Уровень 4. Анализ и визуализация данных на языке Python. Библиотеки numpy, pandas, matplotlib"

## Модуль 5. Библиотека pandas. Работа с датасетами.

- Загрузка датасетов
- Обработка отсутствующих данных
- Поиск и удаление дублей
- Создание новых признаков, функции ```apply()``` и ```applymap()```
- Категориальные признаки, функция ```cut()```, dummy-признаки
- Горизонтальные и вертикальные объединения, функции ```merge()``` и ```concat()```
- "Широкий" и "Длинный" форматы таблиц (stack/unstack)
- Сохранение датасетов



In [2]:
# загрузите необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Загрузка датасетов

Pandas поддерживает загрузку данных из множества источников. Чаще всего придется работать с данными в форматах CSV, XLSX и JSON, а также загружать их из базы данных.

Рассмотрим загрузку данных из файла формата csv - данных, разделенных запятыми. Посмотрим содержимое файла, который мы будем загружать:

In [5]:
with open('data/load_example1.csv') as f:
    print(f.read())

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo
16,,18,18,bar
9,10,11,12,baz
1,2,3,4,hello
0,1,,,xxx
7,6,5,,yyy


Для загрузки будем использовать функцию ```pd.read_csv()```. Укажите в качестве параметра имя файла.

In [6]:
df = pd.read_csv('data/load_example1.csv')
df

,a,b,c,d,message
0,1,2.0,3.0,4.0,hello
1,5,6.0,7.0,8.0,world
2,9,10.0,11.0,12.0,foo
3,16,NaN,18.0,18.0,bar
4,9,10.0,11.0,12.0,baz
5,1,2.0,3.0,4.0,hello
6,0,1.0,NaN,NaN,xxx
7,7,6.0,5.0,NaN,yyy


Обратите внимание, как ведет себя функция по умолчанию:
 - названия колонок соответствуют содержимому первой строки файла
 - индекс по умолчанию - последовательность чисел.
 
Чтобы ```read_csv()``` включила первую строку в наш DataFrame, передайте ей параметр ```header=None```:

In [7]:
df = pd.read_csv('data/load_example1.csv', header=None)
df

,0,1,2,3,4
0,a,b,c,d,message
1,1,2,3,4,hello
2,5,6,7,8,world
3,9,10,11,12,foo
4,16,NaN,18,18,bar
5,9,10,11,12,baz
6,1,2,3,4,hello
7,0,1,NaN,NaN,xxx
8,7,6,5,NaN,yyy


Также можно задать названия столбцов самостоятельно:

In [8]:
df = pd.read_csv('data/load_example1.csv', names=['aa', 'bb', 'cc', 'dd', 'mmessage'])
df

,aa,bb,cc,dd,mmessage
0,a,b,c,d,message
1,1,2,3,4,hello
2,5,6,7,8,world
3,9,10,11,12,foo
4,16,NaN,18,18,bar
5,9,10,11,12,baz
6,1,2,3,4,hello
7,0,1,NaN,NaN,xxx
8,7,6,5,NaN,yyy


Чтобы указать, что один из столбцов - индекс, используйте параметр index_col, там можно указать либо название поля, либо его порядковый номер:

In [9]:
df = pd.read_csv('data/load_example1.csv', index_col='message')
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [10]:
# или 
df = pd.read_csv('data/load_example1.csv', index_col=4)
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


Чтобы пропустить те или иные строки, используйте параметр ```skiprows```, ему можно передать список строк, которые надо пропустить:

In [11]:
df = pd.read_csv('data/load_example1.csv', skiprows=[0,1])
df

,5,6,7,8,world
0,9,10.0,11.0,12.0,foo
1,16,NaN,18.0,18.0,bar
2,9,10.0,11.0,12.0,baz
3,1,2.0,3.0,4.0,hello
4,0,1.0,NaN,NaN,xxx
5,7,6.0,5.0,NaN,yyy


Обратите внимание: указанные строки вообще не участвуют в разборе файла!

При разборе CSV-файлов также могут встретиться следующие трудности:
 - вместо отсутствующих данных могут быть строки типа "NULL", "n/a" и т.п.
 - разделителями могут быть символы ";" (особенно при выгрузке данных из русской версии Microsoft Excel), или же символ табуляции.
 
Со всем этим может справиться функция ```read_csv()```. Загрузим файл ```data/load_example2.csv```

In [12]:
with open('data/load_example2.csv') as f:
    print(f.read())

col1;col2;col3
1;2;3
4;"данные отсутствуют";6
7;8;9
"данные отсутствуют";8;9


 Для указания символа ";" в качестве разделителя, передайте фукнции параметр ```sep=';'```

In [13]:
df = pd.read_csv('data/load_example2.csv', sep=';')
df

,col1,col2,col3
0,1,2,3
1,4,данные отсутствуют,6
2,7,8,9
3,данные отсутствуют,8,9


Чтобы обработать строки "данные отсутствуют" в данном примере, функции ```read_csv()``` нужно передать параметр ```na_values='данные отсутствуют'```

In [14]:
df = pd.read_csv('data/load_example2.csv', sep=';', na_values='данные отсутствуют')
df

,col1,col2,col3
0,1.0,2.0,3
1,4.0,NaN,6
2,7.0,8.0,9
3,NaN,8.0,9


### Обработка отсутствующих данных

С отсутствующими данными в объекте Series можно сдедать следующее:
 - удалить функцией ```.dropna()```
 - заполнить подходящим значением, используя функцию ```.fillna()```.
 
Для поиска пустых значений используем функцию ```.isnull()```.
 
Посмотрим, как это работает на примере первого сета. Снова загрузим его.

In [70]:
df = pd.read_csv('data/load_example1.csv', index_col='message')
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


Получить series из позиций в 'b', содержащих NaN, можно используя булеву маску по колонке "b":

In [16]:
df['b'][df['b'].isnull()]

message
bar   NaN
Name: b, dtype: float64

Посмотрим, как работает ```.dropna()``` в Series, получим колонку 'b' в виде этого объекта:

In [17]:
b = df['b'].copy()
b

message
hello     2.0
world     6.0
foo      10.0
bar       NaN
baz      10.0
hello     2.0
xxx       1.0
yyy       6.0
Name: b, dtype: float64

Вызовем ```dropna()```:

In [18]:
bbd = b.dropna()
bbd

message
hello     2.0
world     6.0
foo      10.0
baz      10.0
hello     2.0
xxx       1.0
yyy       6.0
Name: b, dtype: float64

Заполним отсутствующие значения

In [19]:
# можно заполнить конкретным значением
bbf = b.fillna(0)
bbf

message
hello     2.0
world     6.0
foo      10.0
bar       0.0
baz      10.0
hello     2.0
xxx       1.0
yyy       6.0
Name: b, dtype: float64

In [20]:
# а можно средним по всей Series
bbf = b.fillna(b.mean())
bbf

message
hello     2.000000
world     6.000000
foo      10.000000
bar       5.285714
baz      10.000000
hello     2.000000
xxx       1.000000
yyy       6.000000
Name: b, dtype: float64

В случае с DataFrame это работает похожим образом, только функция удаляет строки, в которых встречается хотя бы одно незаполненное значение:

In [22]:
df

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,NaN,NaN
yyy,7,6.0,5.0,NaN


In [23]:
df.dropna()

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0


Этой функции можно задать порог срабатывания, в зависимости от количества __заполненных подряд значений__ в строке. Например, нам нужно удалить только те строки, в которых заполнены как минимум первые три значения подряд:

In [24]:
df.dropna(thresh=3)

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,NaN,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
yyy,7,6.0,5.0,NaN


Обратите внимание на строку "bar" - несмотря на незаполненную ячейку, она не попала под удаление!

Также можно заполнять отсутствующие данные числами:

In [25]:
df.fillna(100500)

,a,b,c,d
message,,,,
hello,1,2.0,3.0,4.0
world,5,6.0,7.0,8.0
foo,9,10.0,11.0,12.0
bar,16,100500.0,18.0,18.0
baz,9,10.0,11.0,12.0
hello,1,2.0,3.0,4.0
xxx,0,1.0,100500.0,100500.0
yyy,7,6.0,5.0,100500.0


Эта функция работает и для заполнения "пробелов" горизонтальными/вертикальными агрегатными вычислениями.

In [26]:
df.fillna(df.mean())

,a,b,c,d
message,,,,
hello,1,2.000000,3.000000,4.000000
world,5,6.000000,7.000000,8.000000
foo,9,10.000000,11.000000,12.000000
bar,16,5.285714,18.000000,18.000000
baz,9,10.000000,11.000000,12.000000
hello,1,2.000000,3.000000,4.000000
xxx,0,1.000000,8.285714,9.666667
yyy,7,6.000000,5.000000,9.666667


Чтобы эти функции отработали внутри самого объекта и не возвращали его копию, используйте параметр ```inplace=True```.

__ЗАДАНИЕ__ Замените отсутствующие значения в колонке b на среднее по ней, c - на 0, d - на среднее по всей матрице.

In [61]:
df['b'].fillna(df['b'].mean(), inplace=True)
df['c'].fillna(0, inplace=True)
df['d'].fillna(df.mean().mean(), inplace=True)
df

,a,b,c,d
message,,,,
hello,1,2.000000,3.0,4.000000
world,5,6.000000,7.0,8.000000
foo,9,10.000000,11.0,12.000000
bar,16,5.285714,18.0,18.000000
baz,9,10.000000,11.0,12.000000
hello,1,2.000000,3.0,4.000000
xxx,0,1.000000,0.0,7.309524
yyy,7,6.000000,5.0,7.309524


### Поиск и удаление дублей

Проверить, является ли уникальным индекс, можно, опросив свойство индекса ```is_unique```:

In [62]:
df.index.is_unique

False

Получить булеву маску для дубликатов по индексу можно, вызвав метод ```.duplicated()```. Применение отрицания этой маски вернет DataFrame без строки с дублированным индексом.

In [63]:
df.index.duplicated()

array([False, False, False, False, False,  True, False, False])

In [64]:
df[~df.index.duplicated()]

,a,b,c,d
message,,,,
hello,1,2.000000,3.0,4.000000
world,5,6.000000,7.0,8.000000
foo,9,10.000000,11.0,12.000000
bar,16,5.285714,18.0,18.000000
baz,9,10.000000,11.0,12.000000
xxx,0,1.000000,0.0,7.309524
yyy,7,6.000000,5.0,7.309524


Тем же методом объекта DataFrame или Series можно получить булеву маску для дубликатов записей в датасете:

In [65]:
df.duplicated()

message
hello    False
world    False
foo      False
bar      False
baz       True
hello     True
xxx      False
yyy      False
dtype: bool

Методу можно передать параметр ```keep=```, который не будет отмечать признаком True либо первый дубликат (значение first), либо последний (значение last).

In [66]:
df.duplicated(keep='last')

message
hello     True
world    False
foo       True
bar      False
baz      False
hello    False
xxx      False
yyy      False
dtype: bool

Метод можно вызвать, передав ему список признаков, в которм нужно ограничиться поиском дубликатов:

In [67]:
df.duplicated(['b'])

message
hello    False
world    False
foo      False
bar      False
baz       True
hello     True
xxx      False
yyy       True
dtype: bool

Удалить дубликаты можно функцией ```drop_duplicates()```. Она работает так же, как и ```duplicated()```, но она возвращает новый DataFrame без дубликатов. Ее можно вызвать с параметром inplace().

In [68]:
df.drop_duplicates()

,a,b,c,d
message,,,,
hello,1,2.000000,3.0,4.000000
world,5,6.000000,7.0,8.000000
foo,9,10.000000,11.0,12.000000
bar,16,5.285714,18.0,18.000000
xxx,0,1.000000,0.0,7.309524
yyy,7,6.000000,5.0,7.309524


In [75]:
df.iloc[5, 1] = 1000
df.duplicated()
#df

message
hello    False
world    False
foo      False
bar      False
baz       True
hello    False
xxx      False
yyy      False
dtype: bool

### Создание новых признаков, функции apply() и applymap()

С созданием новых признаков на базе существующих данных мы уже знакомы, но часто бывает так, что для вычисления новых признаков нужно применить более сложные процедуры, чем стандартные. Для этого существуют функции ```apply()``` и ```applymap()```.



In [76]:
df = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df

,b,d,e
Utah,0.650187,0.300264,-0.994349
Ohio,1.845390,-0.929671,-0.257088
Texas,-1.305679,0.507673,-1.201291
Oregon,0.165051,1.409329,1.319838


Посмотрим, как работает метод ```apply()```. Функция, которая указана в качестве параметра этого метода принимает на вход объект Series - столбец и возвращает значение, которое объединяется в объект Series, структцрно соответствующий строке текущего DataFrame. Для вычисления по строкам и формирования столбцов функции ```apply()``` нужно передать параметр ```axis=1```

In [79]:
# Функции f и ff эквивалентны:
def f(x):
    print(x)
    return x.max() - x.min()

ff = lambda x: x.max() - x.min()

df.apply(f, axis=1)

b    0.650187
d    0.300264
e   -0.994349
Name: Utah, dtype: float64
b    1.845390
d   -0.929671
e   -0.257088
Name: Ohio, dtype: float64
b   -1.305679
d    0.507673
e   -1.201291
Name: Texas, dtype: float64
b    0.165051
d    1.409329
e    1.319838
Name: Oregon, dtype: float64


Utah      1.644536
Ohio      2.775060
Texas     1.813351
Oregon    1.244278
dtype: float64

Добавление нового вычисленного признака теперь будет выглядеть так:

In [81]:
df['diff'] = df.apply(ff, axis=1)
df

,b,d,e,diff
Utah,0.650187,0.300264,-0.994349,2.638885
Ohio,1.845390,-0.929671,-0.257088,3.704731
Texas,-1.305679,0.507673,-1.201291,3.119030
Oregon,0.165051,1.409329,1.319838,1.244278


В отличие от ```apply()```, ```applymap()``` вычисляется для каждого элемента и возвращает значение, которое должно быть установлено на его место.

In [82]:
format_ = lambda x: '%.2f' % x
df.applymap(format_)

,b,d,e,diff
Utah,0.65,0.30,-0.99,2.64
Ohio,1.85,-0.93,-0.26,3.70
Texas,-1.31,0.51,-1.20,3.12
Oregon,0.17,1.41,1.32,1.24


Для того, чтобы проделать такую операцию над Series, воспользуйтесь функцией map():

In [83]:
# df['e'] = df['e'].map(format_)
df
df['e'] = df['e'].map(lambda x: float(x))

__ЗАДАНИЕ__ В текущий DataFrame ```df``` добавьте строку с суммами значений 1000, если значение больше нуля, и 0 в противном случае.

In [106]:
replace = lambda x: 1000 if x > 0 else 0
#df.append(df.applymap(replace).sum(), ignore_index=True)


df.append(pd.Series(df.applymap(lambda x: 1000 if x > 0 else 0).sum(), name='Sum'))


,b,d,e,diff
Utah,0.650187,0.300264,-0.994349,2.638885
Ohio,1.845390,-0.929671,-0.257088,3.704731
Texas,-1.305679,0.507673,-1.201291,3.119030
Oregon,0.165051,1.409329,1.319838,1.244278
Sum,3000.000000,3000.000000,1000.000000,4000.000000


__ЗАДАНИЕ__  В датасете Titanic проверьте признак "Возраст"("Age") на выбросы (отрицательный возраст, посмотрите максимальный возраст - он правдоподобен?).
Если там есть отсутствующие значения - на их место поставьте медианный возраст пассажиров.

In [123]:
# загрузите Titanic
df_titanic = pd.read_csv('data/titanic.csv',
                  index_col='PassengerId')

#df_titanic[df_titanic['Age'] < 1]
print(df_titanic['Age'].describe())
df_titanic['Age'].isnull().sum()
df_titanic['Age'].fillna(df_titanic['Age'].median(), inplace=True)
print(df_titanic['Age'].describe())

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64
count    891.000000
mean      29.361582
std       13.019697
min        0.420000
25%       22.000000
50%       28.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64


### Категориальные признаки, функция cut(), dummy-признаки

Часто возникает задача сделать более точным один из признаков, сократив по нему количество возможных вариантов, а то и вообще сведя к одному или нескольким булевам признакам (dummy-признакам).

Это может быть применено к различным количественным характеристикам (например, возраст, вес - "несовершеннолетний"/"толстый"), к географическим признакам ("Москва"/"не Москва"), к временным признакам ("До Революции/После Революции") и т.д.

Рассмотрим создание категориальных признаков на примере работы с датасетом "Титаник".

**Создадим признак "Возрастная категория"**

Создавать будем двумя способами: 
1. с помощью функции, которая возвращает 1, если до 30-ти, 2, если от 30-ти до 55-ти и 3, если старше 55.
2. с помощью функции ```pd.cut()```

In [126]:
def age_category(age):
    '''
    < 30 -> 1
    >= 30, <55 -> 2
    >= 55 -> 3
    '''
    if age < 30:
        return 1
    elif age < 55:
        return 2
    else:
        return 3
    
df_titanic['Age_category'] = df_titanic['Age'].apply(age_category)
df_titanic

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_category
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,2
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,NaN,S,1


Теперь функцией ```cut()```:

In [127]:
# создадим "козрзинки", в которые будем раскладывать наши категории
bins = [0,30,55,100]
age_categories = pd.cut(df_titanic['Age'], bins, right=False) # right=False - означает, что правая граница НЕ включена
age_categories

PassengerId
1       [0, 30)
2      [30, 55)
3       [0, 30)
4      [30, 55)
5      [30, 55)
         ...   
887     [0, 30)
888     [0, 30)
889     [0, 30)
890     [0, 30)
891    [30, 55)
Name: Age, Length: 891, dtype: category
Categories (3, interval[int64]): [[0, 30) < [30, 55) < [55, 100)]

Чтобы добавить требуемые метки, передадим их в виде списка:

In [128]:
labels = [1,2,3]
age_categories = pd.cut(df_titanic['Age'], bins, labels=labels, right=False) 
age_categories

PassengerId
1      1
2      2
3      1
4      2
5      2
      ..
887    1
888    1
889    1
890    1
891    2
Name: Age, Length: 891, dtype: category
Categories (3, int64): [1 < 2 < 3]

Теперь добавим их в наш датасет и сравним с тем, что мы сделали с помощью функции ```apply()```:

In [129]:
df_titanic['Age_category_1'] = pd.cut(df_titanic['Age'], bins, labels=labels, right=False)

In [130]:
df_titanic.T.duplicated()

Survived          False
Pclass            False
Name              False
Sex               False
Age               False
SibSp             False
Parch             False
Ticket            False
Fare              False
Cabin             False
Embarked          False
Age_category      False
Age_category_1     True
dtype: bool

Еще раз посмотрим на добавленные признаки:

In [131]:
df_titanic[ ['Age_category', 'Age_category_1']]

,Age_category,Age_category_1
PassengerId,,
1,1,1
2,2,2
3,1,1
4,2,2
5,2,2
...,...,...
887,1,1
888,1,1
889,1,1


In [132]:
df_titanic['Age_category_1'][0:10]

PassengerId
1     1
2     2
3     1
4     2
5     2
6     1
7     2
8     1
9     1
10    1
Name: Age_category_1, dtype: category
Categories (3, int64): [1 < 2 < 3]

Тот признак, который мы создали из функции ```cut()``` стал категориальным - его значения могут принимать три величины: 1, 2 или 3.

#### Добавление dummy-признаков

В данном случае мы вместо одного признака "возрастная категория" с тремя возможными значениями сделаем три булевых признака. В задачах машинного обучения бывает необходимость оценить степень влияния принадлежности к той или иной группе на решение задачи, и если влияние незначительное - избавиться от такого признака. Потом, ряд алгоритмов принимает на вход только цифровые значения, и такое действие позволяет избавиться от one-hot encoding для таких признаков.

Добавить их можно очень просто: функцией ```pd.get_dummies()```. При этом признак, из которого мы получаем эти dummy-признаки, не обязательно должен быть категориальным.

In [133]:
age_dummies = pd.get_dummies(df_titanic['Age_category'])
age_dummies

,1,2,3
PassengerId,,,
1,1,0,0
2,0,1,0
3,1,0,0
4,0,1,0
5,0,1,0
...,...,...,...
887,1,0,0
888,1,0,0
889,1,0,0


Как видно, названия колонок для этих признаков взяты из их значений. Чтобы придать им осмысленное название, пользуйтесь параметром ```prefix=```.

In [135]:
age_dummies = pd.get_dummies(df_titanic['Age_category_1'], prefix="age_cat")
age_dummies

,age_cat_1,age_cat_2,age_cat_3
PassengerId,,,
1,1,0,0
2,0,1,0
3,1,0,0
4,0,1,0
5,0,1,0
...,...,...,...
887,1,0,0
888,1,0,0
889,1,0,0


Присоединить наши новые признаки к датасету можно методом ```.join()```.

In [136]:
df_titanic = df_titanic.join(pd.get_dummies(df_titanic['Age_category_1'], prefix="age_cat_"))

In [137]:
df_titanic

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_category,Age_category_1,age_cat__1,age_cat__2,age_cat__3
PassengerId,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,1,1,0,0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,2,0,1,0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,1,0,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,2,0,1,0
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,2,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1,1,1,0,0
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,1,1,0,0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,NaN,S,1,1,1,0,0


### Горизонтальные и вертикальные объединения, функции merge() и concat()

"Горизонтальные" объединения (аналог JOIN в SQL) в pandas выполняются функцией или методом ```merge()```. По умолчанию оъединение производится по колонкам с совпадающими именами и только по ключам, которые включаются в оба DataFrame'а.

Создадим DataFrame с номерами грузовиков и некоторой абстрактной статистикой по ним.

In [138]:
trucks = ['X101AP', 'T123TM', 'X098AP', 'T123TM',  'X098AP', 'X101AP']
df_trucklog = pd.DataFrame({'truck':trucks, 'week':[12,10,5,6,7,9], 'month':[212,310,85,186,217,299]}, columns=['truck', 'week', 'month'])
df_trucklog

,truck,week,month
0,X101AP,12,212
1,T123TM,10,310
2,X098AP,5,85
3,T123TM,6,186
4,X098AP,7,217
5,X101AP,9,299


И создадим DataFrame со справочником по этим грузовикам, которые включают, например, марку. 

In [139]:
df_trucks = pd.DataFrame({'plate_number': df_trucklog['truck'].unique(),
'brand': ['VOLVO', 'RENAULT', 'MAN']}, columns=['plate_number', 'brand'])
df_trucks

,plate_number,brand
0,X101AP,VOLVO
1,T123TM,RENAULT
2,X098AP,MAN


Предположим, заказчику захотелось увидеть в отчете по этим грузовикам не только номер, но и марку, а в изначальном датасете она отсутствует. Мы можем "вытащить" марку из справочника, выполнив функцию ```merge()```.

Укажем в параметрах названия полей, по которым надо выполнить объединение. В результате будет возвращен новый DataFrame.

In [140]:
df_trucklog.merge(df_trucks, left_on='truck', right_on='plate_number')

,truck,week,month,plate_number,brand
0,X101AP,12,212,X101AP,VOLVO
1,X101AP,9,299,X101AP,VOLVO
2,T123TM,10,310,T123TM,RENAULT
3,T123TM,6,186,T123TM,RENAULT
4,X098AP,5,85,X098AP,MAN
5,X098AP,7,217,X098AP,MAN


Добавим в журнал по грузовикам машину, которой нет в справочнике.

In [141]:
df_trucklog = df_trucklog.append({'week': 5, 'month': 20, 'truck':'X055XT'}, ignore_index=True)
df_trucklog

,truck,week,month
0,X101AP,12,212
1,T123TM,10,310
2,X098AP,5,85
3,T123TM,6,186
4,X098AP,7,217
5,X101AP,9,299
6,X055XT,5,20


Если мы хотим, чтобы данные по этой машине также присутствовали в отчете, мы можем включить все ключи слева параметром ```how='left'```.

In [142]:
df_trucklog.merge(df_trucks, left_on='truck', right_on='plate_number', how='left')

,truck,week,month,plate_number,brand
0,X101AP,12,212,X101AP,VOLVO
1,T123TM,10,310,T123TM,RENAULT
2,X098AP,5,85,X098AP,MAN
3,T123TM,6,186,T123TM,RENAULT
4,X098AP,7,217,X098AP,MAN
5,X101AP,9,299,X101AP,VOLVO
6,X055XT,5,20,NaN,NaN


"Вертикальное" объединение таблиц возможно с помощью функции ```concat()```. На вход она получает список датафреймов, которые надо объединить. Если вы используете сгенерированные ключи, не забудьте указать параметр ```ignore_keys=True```.

In [143]:
df_trucklog1 = pd.DataFrame({'truck':trucks, 'week':[2,7,6,6,2,1], 'month':[50,25,110,162,272,292]}, columns=['truck', 'week', 'month'])
df_trucklog1

,truck,week,month
0,X101AP,2,50
1,T123TM,7,25
2,X098AP,6,110
3,T123TM,6,162
4,X098AP,2,272
5,X101AP,1,292


In [144]:
df_trucklog_new = pd.concat([df_trucklog, df_trucklog1], ignore_index=True)
df_trucklog_new

,truck,week,month
0,X101AP,12,212
1,T123TM,10,310
2,X098AP,5,85
3,T123TM,6,186
4,X098AP,7,217
5,X101AP,9,299
6,X055XT,5,20
7,X101AP,2,50
8,T123TM,7,25
9,X098AP,6,110


__???__ А как разбить DataFrame? 

__ЗАДАНИЕ__

Есть таблица студентов и номеров их зачетных книжек. Есть несколько объектов Series с оценками по различным предметам, где индексы - номера зачетных книжек. Нужно получить следующие данные:
1. Получить объединенный табель по всем предметам и студентам.
2. Получить список студентов, сдавших сессию на "хорошо" и "отлично"
3. Получить список студентов, которые сдали не все экзамены

In [193]:
df_students = pd.DataFrame({'surname': ['Ivanov', 'Petrov', 'Sidorov', 'Kuznetsov', 'Kotova', 'Ivanov'],\
                           'logbook': ['X01', 'X02', 'X04', 'X03', 'X05', 'X05', ]})
s_physics = pd.Series([5,5,2,3,4], index=['X05', 'X02', 'X03', 'X06', 'X01', ])
s_calculus = pd.Series([4,3,5,5,4,5], index=['X02', 'X01', 'X04', 'X05', 'X06', 'X03'])
s_linalg = pd.Series([5,2,3,4], index=['X01', 'X03', 'X05', 'X06'])

df_scores = pd.DataFrame( {'phys': s_physics, 'calc': s_calculus, 'lalg': s_linalg })
df = df_students.merge(df_scores, left_on='logbook', right_index=True, how='left')
df[(df[ df_scores.columns ] >= 4).prod(axis=1).astype(dtype=bool)]
print(df)
#df[df[ df_scores.columns ].isnull()]
mask = df[ df_scores.columns ].isnull().any(axis=1)
df[mask]
#print(mask)


     surname logbook  phys  calc  lalg
0     Ivanov     X01   4.0     3   5.0
1     Petrov     X02   5.0     4   NaN
2    Sidorov     X04   NaN     5   NaN
3  Kuznetsov     X03   2.0     5   2.0
4     Kotova     X05   5.0     5   3.0
5     Ivanov     X05   5.0     5   3.0


,surname,logbook,phys,calc,lalg
1,Petrov,X02,5.0,4,NaN
2,Sidorov,X04,NaN,5,NaN


### "Раскатка" и "Штабелирование" данных

Работа со сложным индексм: функции ```stack()``` и ```unstack()```.

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data


 Функция ```stack()``` "сложит" данные вертикально, построив "сложный индекс":

In [ ]:
stacked = data.stack()
stacked

Функция ```unstack()``` выполнит противоположную задачу - данные из "сложного индекса" вынесет в колонки:

In [ ]:
stacked.unstack()

#### Функции melt() и pivot()

```melt()``` - преобразует столбцы в строки, добавляя соотвествующие столбцы variable и value. \
```pivot()```- наоборот, собирает данные по строкам в столбцы

In [2]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [4]:
melted = pd.melt(df, ['key'], var_name='letters', value_name='numbers')
melted

,key,letters,numbers
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [6]:
reshaped = pd.pivot(melted, index=['key'], columns=['letters'], values='numbers')
reshaped

letters,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


#### Сохранение данных

Для сохранения датасетов в требуемом виде можно использовать следующие функции:
 - ```to_csv()``` - для сохранения данных в виде CSV
 - ```to_excel()``` - для сохранения данных в виде Excel Workbook \
...а также во можестве других форматов (см. документацию).
 
Также данные можно экспортировать в структуры Python и numpy:
 - ```to_dict()``` - этот метод вернет словарь с содержимым DataFrame, ровно в том же виде, чтобы из него можно было бы создать новый DataFrame
 - ```to_dict('records')``` - в данном случае этот метод вернет список словарей
 - ```to_numpy()``` - а этот метод можно использовать, если вам нужна матрица numpy

__ЗАДАНИЕ__: Загрузите датасет "toy_budget.csv". Он содержит информацию по доходам и расходам подразделений компании по месяцам. Преобразуйте его в датасет, в котором месяцы отложены как столбцы и упорядочены так, как это задано в списке ```months```. Подсчитайте суммы по каждому месяцу.

Указание: используйте функцию ```reindex()``` для упорядочивания колонок.

In [14]:
months = ['apr',
 'may',
 'jun',
 'jul',
 'aug',
 'sep',
 'oct',
 'nov',
 'dec',
 'jan',
 'feb',
 'mar',]

df_toy_budget = pd.read_csv('data/toy_budget.csv', sep=';', index_col=0)
df_toy_budget

,Div,Account,Type,Month,Amount
0,IT,System Costs,Cost,apr,364.68
1,IT,IT Support,Cost,apr,62.27
2,Finance,Accounting,Cost,apr,78.89
3,Finance,Billing,Cost,apr,55.81
4,Adm,Office Adm,Cost,apr,88.94
...,...,...,...,...,...
103,Adm,Office Adm,Cost,mar,88.94
104,Air,Gross Margin,Income,mar,1055.67
105,Truck,Gross Margin,Income,mar,931.48
106,Rail,Gross Margin,Income,mar,919.21


In [ ]:
# ваш код здесь